<a href="https://colab.research.google.com/github/descandon88/tesis_maestria/blob/main/analisis_mozambique/feasible_products_proposition_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
# Librerías para visualizacion
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import re
import networkx as nx
from itertools import count
from itertools import combinations
from itertools import product
import statsmodels.api as sm
import urllib.request, json
!pip3 install geopandas
import geopandas
# -- set scientific notation to display numbers fully rather than exponential
pd.set_option('display.float_format', '{:.2f}'.format)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' 
import seaborn as sns
sns.set_style('whitegrid') # grids en dark background

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth',300)
print('librerías cargadas')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.1 MB/s eta 0:00:00
librerías cargadas


### Importacion de la bases desde drive



Se vincula la carpata de las bases con drive

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
## Se importa la base de exportaciones de productos drive
df_base = pd.read_csv("/content/drive/MyDrive/eci/base/df_updated.csv")
df_short_name = pd.read_csv("/content/drive/MyDrive/eci/base/df_tipo_pro.csv")

print('dataframes cargados')

dataframes cargados


In [4]:
df_base.describe()

,year,product_code,export_value
count,7373271.00,7373271.00,7373271.00
mean,2007.65,5304.58,42867657.55
std,7.44,2767.02,899054286.46
min,1995.00,101.00,0.00
25%,2001.00,2843.00,0.00
50%,2008.00,5402.00,1119.00
75%,2014.00,8102.00,432040.50
max,2020.00,9999.00,371679199232.00


In [5]:
# df_base['product_code'] = df_base['product_code'].apply(lambda x: '{0:0>4}'.format(x))
# df['product_code']  = df['product_code'] .apply(np.int64)

df_base.sample(n=6)
df_base.info()

,year,country_code,country_name,product_code,product_name,export_value
2712206,2011,SAU,Saudi Arabia,3913,"Natural polymers (eg alginic acid) and modified natural polymers (eg hardened proteins, chemical derivatives of natural rubber), n.e.s. or included, in primary forms",1114226
4410875,2004,HTI,Haiti,6702,"Flowers, foliage and fruit, artificial, and parts thereof; articles made of artificial flowers, foliage or fruit",0
7112439,1999,UZB,Uzbekistan,9302,Revolvers and pistols; other than those of heading no. 9303 or 9304,0
5895940,2016,DNK,Denmark,8425,Pulley tackle and hoists other than skip hoists; winches and capstans; jacks,71447352
6186002,2014,VNM,Vietnam,8473,"Machinery; parts and accessories (not covers, carrying cases and the like) suitable for use solely, principally with machines of heading no. 8469 to 8472",589140224
5162322,2007,MSR,Montserrat,7402,"Copper; unrefined, copper anodes for electrolytic refining",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7373271 entries, 0 to 7373270
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   year          int64 
 1   country_code  object
 2   country_name  object
 3   product_code  int64 
 4   product_name  object
 5   export_value  int64 
dtypes: int64(3), object(3)
memory usage: 337.5+ MB


####Finding specific countries / products based on partial string matching If you’re interested in finding data on certain countries / products but not sure how exactly these are spelled in the data (or are spelled with / without e.g. capital letters)

In [6]:
STRING = 'Netherland'
df_base[df_base['country_name'].str.contains(STRING)][['country_name']].drop_duplicates()

STRING = 'wine'
df_base[df_base['product_name'].str.contains(STRING,flags=re.IGNORECASE, regex=True)][['product_name']].drop_duplicates()

,country_name
156,Netherlands Antilles
3711,Netherlands


,product_name
11924,Swine; live
47696,"Meat of swine; fresh, chilled or frozen"
65582,"Edible offal of bovine animals, swine, sheep, goats, horses, asses, mules or hinnies; fresh, chilled or frozen"
1084954,"Wine of fresh grapes, including fortified wines; grape must other than that of heading no. 2009"
1090916,"Vermouth and other wine of fresh grapes, flavoured with plants or aromatic substances"
1156498,Wine lees; argol
3855058,"Twine, cordage, ropes and cables, whether or not plaited or braided; whether or not impregnated, coated, covered or sheathed with rubber or plastics"
3861020,"Twine, cordage or rope; knotted netting, made up fishing nets and other made up nets, of textile materials"
3866982,"Articles of yarn, strip or the like of heading no. 5404 or 5405; twine, cordage, rope or cables n.e.s. or included"
4302208,"Rags; used or new, scrap twine, cordage, rope and cables and worn out articles of twine, cordage, rope or cables, of textile materials"


### Cálculo de Complejidad Económica

In [7]:
!pip install ecomplexity
print('py-ecomplexity instalado')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ecomplexity: filename=ecomplexity-0.5.2-py3-none-any.whl size=11283 sha256=e3df97802a4752da67f36383e85242bc53176234d40b56414cbd293eee02b16a
  Stored in directory: /root/.cache/pip/wheels/c7/14/29/d28c7286333b75599f8f60681634819c63330cc3b89cf85f61
Successfully built ecomplexity
py-ecomplexity instalado


In [8]:
from ecomplexity import ecomplexity
from ecomplexity import proximity

# To use py-ecomplexity, specify the following columns
# cols_input: dict of column names for time, location, product and value.
    #Example: {'time':'year', 'loc':'origin', 'prod':'hs92', 'val':'export_val'}
    
trade_cols = {'time':'year',
              'loc':'country_name',
              'prod':'product_name',
              'val':'export_value'}

print('Calculando complejidad económica')
df_ec = ecomplexity(df_base, trade_cols)
print('Cálculo finalizado')



Calculando complejidad económica
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Cálculo finalizado


In [9]:
df_complexity = df_ec[['country_name',
               'product_name',
               'product_code',
               'export_value',
               'rca',
               'mcp',
               'density',
               'year',
               'pci',
               'eci',
               'coi',
               'cog']].copy()

In [10]:
# Muestro resultados
df_complexity.head(n=8)

,country_name,product_name,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog
0,Afghanistan,"Abrasive powder or grain; natural or artificial, on a base of textile material, of paper, paperboard or of other material, whether or not cut to shape or sewn or otherwise made up",6805,1327,0.05,0,0.04,1995,3.01,-0.33,-0.59,1.37
1,Afghanistan,Accordions and similar instruments; mouth organs,9204,0,0.00,0,0.03,1995,3.88,-0.33,-0.59,1.05
2,Afghanistan,"Acetals and hemiacetals; whether or not with other oxygen function, and their halogenated, sulphonated, nitrated or nitrostated derivatives",2911,0,0.00,0,0.03,1995,3.77,-0.33,-0.59,1.42
3,Afghanistan,"Acids; carboxylic acid with additional oxygen function and their anhydrides, halides, peroxides, peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2918,0,0.00,0,0.04,1995,2.37,-0.33,-0.59,0.99
4,Afghanistan,"Acids; polycarboxylic acids, their anhydrides, halides, peroxides and peroxy-acids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2917,0,0.00,0,0.05,1995,1.50,-0.33,-0.59,0.74
5,Afghanistan,"Acids; saturated acyclic monocarboxylic acids and their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2915,0,0.00,0,0.03,1995,3.03,-0.33,-0.59,1.30
6,Afghanistan,"Acids; unsaturated acyclic monocarboxylic, cyclic monocarboxylic , their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2916,0,0.00,0,0.03,1995,3.33,-0.33,-0.59,1.51
7,Afghanistan,Acrylic polymers in primary forms,3906,0,0.00,0,0.03,1995,3.76,-0.33,-0.59,1.19


### Se filtran los años de la Preoposition #3

In [ ]:
# years=[2014,2015,2016,2017,2018]
# prepoOneYears= df_complexity[df_complexity['year'].isin(years)].copy()
# # prepoOneYears.head()


### Se realizan las siguientes tablas:


1.   Tabla con la distancia de productos para el 2014
2.   Tabla de rca de Mozambique para el 2014
3.   Tabla de OGI en el 2014




#### Cálculos de distancias(2014) y cálculos de rca(2014)

In [85]:
prepoThree2014 = df_complexity[df_complexity['year']==2014].copy()
# rcaMoz_2014 =  df_complexity[(df_complexity['year']==2014)&(df_complexity['country_name']=='Mozambique')].copy()

prepoThree2014 = prepoThree2014[['country_name','product_name','product_code','density','rca','cog','pci','year']].copy()

def Dexport_rca_mozam(data):
    df_all = data.copy()
    # df_all['rca_Mozambique_2014'] = df_all.apply(lambda x: (x.rca),axis =1 )
    df_all['rca_2014'] = df_all.apply(lambda x: (x.rca),axis =1 )

    df_all['Dexport_dummy'] = df_all.apply(lambda x: 1 if (x.rca_2014>0.2 and x.rca_2014 <1) else 0 ,axis=1)

    df_all.drop(['product_name','density','eci','coi'],axis=1,inplace=True,errors='ignore')
    df_all = df_all.reset_index(drop=True)

    print("tabla filtrada!")
    return df_all


def distance_calc(data):
  df_dist = data.copy()
  df_dist['distancia_2014'] = df_dist.apply(lambda x: (1-x.density),axis =1 )
  df_dist['rca_2014'] = df_dist.apply(lambda x: x.rca,axis =1 )
  df_dist['pci_2014'] = df_dist.apply(lambda x: x.pci,axis =1 )


  df_dist.drop(['product_name','year'],axis=1,inplace=True,errors='ignore')
  df_dist = df_dist.reset_index(drop=True)
  print("Calculo de distancia terminado!")
  return df_dist

# t_rcaMozam_2014 = Dexport_rca_mozam(rcaMoz_2014)
t_rca_2014 = Dexport_rca_mozam(prepoThree2014)
t_Distance2014 = distance_calc(prepoThree2014)

# t_rcaMozam_2014.head()
t_rca_2014.head()
t_Distance2014.head()

# t_Distance2014['product_code'].nunique()
# rcaMozam_2014['country_name'].nunique()

count_ones = (t_rca_2014['Dexport_dummy'] == 1).sum()
count_zeros = (t_rca_2014['Dexport_dummy'] == 0).sum()
# count_zeros_export_value = (t_rca_2014['export_value'] == 0).sum()

tabla filtrada!
Calculo de distancia terminado!


,country_name,product_code,rca,cog,pci,year,rca_2014,Dexport_dummy
0,Afghanistan,6805,0.00,1.40,3.72,2014,0.00,0
1,Afghanistan,2911,0.00,1.01,1.89,2014,0.00,0
2,Afghanistan,2918,0.00,0.79,1.99,2014,0.00,0
3,Afghanistan,2917,0.00,0.73,2.34,2014,0.00,0
4,Afghanistan,2915,0.00,0.98,3.05,2014,0.00,0


,country_name,product_code,density,rca,cog,pci,distancia_2014,rca_2014,pci_2014
0,Afghanistan,6805,0.04,0.00,1.40,3.72,0.96,0.00,3.72
1,Afghanistan,2911,0.03,0.00,1.01,1.89,0.97,0.00,1.89
2,Afghanistan,2918,0.05,0.00,0.79,1.99,0.95,0.00,1.99
3,Afghanistan,2917,0.04,0.00,0.73,2.34,0.96,0.00,2.34
4,Afghanistan,2915,0.04,0.00,0.98,3.05,0.96,0.00,3.05


In [86]:

count_ones = (t_rca_2014['Dexport_dummy'] == 1).sum()
count_zeros = (t_rca_2014['Dexport_dummy'] == 0).sum()
# count_zeros_export_value = (t_rca_2014['export_value'] == 0).sum()

print("cantidad de productos en el 2014:",t_Distance2014['product_code'].nunique())
print("cantidad de países en el 2014:",t_Distance2014['country_name'].nunique()) 
# print("cantidad de países en t_rcaMozam_2014:",t_rcaMozam_2014['country_name'].nunique())
print("cantidad de años en  t_rca_2014:",t_rca_2014['year'].nunique())
print("cantidad de productos en  t_rca_2014:",t_rca_2014['product_code'].nunique())
#print("cantidad de export_value en  t_rca_2014:",t_rca_2014['export_value'].nunique())
# print("cantidad de ceros en export_value en  t_rca_2014:",count_zeros_export_value)




print("Count of 1 (ones) in column: Dexport_dummy:", count_ones)
print("Count of 0 (zeros) in column: Dexport_dummy:", count_zeros)

cantidad de productos en el 2014: 1236
cantidad de países en el 2014: 235
cantidad de años en  t_rca_2014: 1
cantidad de productos en  t_rca_2014: 1236
Count of 1 (ones) in column: Dexport_dummy: 37702
Count of 0 (zeros) in column: Dexport_dummy: 252758


### Cálculo de OGI (2014)

In [54]:
def cal_distancia_normalizada(data,
    ):
  "En la siguiente función se realizará el cálculo de COI para cada país del dataframe"
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de distancia...")

  # t_df['density_r'] = t_df[['Mcp','density']].apply(lambda x: 0 if (x.Mcp==0) else x.density,axis=1 )

  # t_df['distancia']=t_df.apply( lambda x: (1-x.density),axis = 1)
  mean =   t_df['distancia_2014'].mean()
  std =   t_df['distancia_2014'].std()

  t_df['dist_2014_norm']=t_df.apply( lambda x: (x.distancia_2014-mean)/std,axis = 1)

  
  return t_df

def calc_PCI_normalizado(data,
    ):
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades

  mean =   t_df['pci'].mean()
  std =   t_df['pci'].std()
  t_df['pci_norm']=t_df.apply( lambda x: ((x.pci-mean))/std,axis = 1)

  print("Se realiza esuma el right con el COG.")

  return t_df


def calc_densitynormalizado(data,
    ):
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de lado derecho...")

  mean =   t_df['density'].mean()
  std =   t_df['density'].std()
  t_df['density_norm']=t_df.apply( lambda x: ((x.density-mean))/std,axis = 1)

  print("Density normalizado! ")

  return t_df

In [146]:
def cal_ogi_2014(data,
    ):
  "En la siguiente función se realizará el cálculo de COI para cada país del dataframe"
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de lado derecho...")

  # t_df['density_r'] = t_df[['Mcp','density']].apply(lambda x: 0 if (x.Mcp==0) else x.density,axis=1 )

  # t_df['right']=t_df.apply( lambda x: (1-x.dist_norm)*x.pci_norm,axis = 1)
  t_df['right']=t_df.apply( lambda x: (x.density)*x.pci,axis = 1)
  # t_df['right']=t_df.apply( lambda x: (1-x.dist_2014_norm)*x.pci,axis = 1)
  

  densityMean = t_df['density'].mean()
  densityStd = t_df['density'].std()
  t_df['density_norm']=t_df.apply( lambda x: (x.density - densityMean)/densityStd,axis = 1)

  pciMean = t_df['pci'].mean()
  pciStd = t_df['pci'].std()

  t_df['pci_norm']=t_df.apply( lambda x: (x.pci - pciMean)/pciStd,axis = 1)


  t_df['right_Nr']=t_df.apply( lambda x: (x.density_norm)*x.pci_norm,axis = 1)



  mean = t_df['right'].mean()
  std = t_df['right'].std()

  meanCog = t_df['cog'].mean()
  stdCog= t_df['cog'].std()

  print("Se realiza la suma el right con el COG.")
  t_df['cog_norm']=t_df.apply( lambda x: (x.cog-meanCog)/stdCog,axis = 1)

  t_df['right_norm']=t_df.apply( lambda x: (x.right-mean)/std,axis = 1)

  t_df['ogi_2014'] = t_df.apply(lambda x: x.cog + x.right, axis=1)

  t_df['ogi_2014_v2'] = t_df.apply(lambda x: x.cog + x.right_Nr, axis=1)

  meanOgi1 =  t_df['ogi_2014'].mean()
  meanOgi2 =  t_df['ogi_2014_v2'].mean()

  stdOgi1 =  t_df['ogi_2014'].std()
  stdOgi2 =  t_df['ogi_2014_v2'].std()

  t_df['ogi_2014_norm'] = t_df.apply(lambda x: (x.ogi_2014 -meanOgi1)/stdOgi1, axis=1)
  t_df['ogi_2014_norm_v2'] = t_df.apply(lambda x: (x.ogi_2014_v2 -meanOgi2)/stdOgi2, axis=1)

  # t_df['ogi2'] = t_df.apply(lambda x: x.cog_norm + x.right_norm, axis=1)
  # meanOgi =  t_df['ogi2'].mean()
  # stdOgi =  t_df['ogi2'].std()
  
  # t_df['ogi2_norm'] = t_df.apply(lambda x: (x.ogi2 -meanOgi)/stdOgi, axis=1)

  t_df.drop(['right_norm','density_norm'],axis=1,inplace=True,errors='ignore')
  print('Cálculo terminado!!')

  return t_df

In [147]:
t_distancia_normalizada = cal_distancia_normalizada(t_Distance2014).copy()

t_ogi2014 = cal_ogi_2014(t_distancia_normalizada).copy()


Se realiza el cálculo de distancia...
Se realiza el cálculo de lado derecho...
Se realiza la suma el right con el COG.
Cálculo terminado!!


In [148]:
t_ogi2014.sample(n=5)
t_ogi2014.describe()

,country_name,product_code,density,rca,cog,pci,distancia_2014,rca_2014,pci_2014,dist_2014_norm,right,pci_norm,right_Nr,cog_norm,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2
262146,Turkey,3002,0.26,0.11,0.92,3.23,0.74,0.11,3.23,-1.31,0.84,1.32,1.73,1.14,1.77,2.65,2.07,2.28
119274,Iceland,8457,0.04,0.00,1.24,5.16,0.96,0.00,5.16,0.63,0.20,2.43,-1.54,1.84,1.43,-0.30,1.53,-0.62
110827,Guinea-Bissau,701,0.01,0.00,-0.21,-0.88,0.99,0.00,-0.88,0.91,-0.01,-1.05,0.95,-1.40,-0.22,0.74,-1.18,0.40
165111,Moldova,1207,0.25,1.03,-0.00,-3.64,0.75,1.03,-3.64,-1.19,-0.90,-2.64,-3.14,-0.92,-0.90,-3.14,-2.30,-3.42
192029,Pakistan,3214,0.17,0.08,0.87,2.17,0.83,0.08,2.17,-0.48,0.36,0.71,0.34,1.03,1.23,1.21,1.20,0.87


,product_code,density,rca,cog,pci,distancia_2014,rca_2014,pci_2014,dist_2014_norm,right,pci_norm,right_Nr,cog_norm,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2
count,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00
mean,5301.76,0.11,1.61,0.41,0.94,0.89,1.61,0.94,0.00,0.09,0.00,-0.08,-0.00,0.50,0.33,-0.00,0.00
std,2764.80,0.12,63.71,0.45,1.74,0.12,63.71,1.74,1.00,0.29,1.00,0.99,1.00,0.61,1.02,1.00,1.00
min,101.00,0.00,0.00,-0.79,-4.76,-0.00,0.00,-4.76,-7.73,-2.22,-3.29,-11.66,-2.70,-2.34,-11.66,-4.66,-11.80
25%,2843.75,0.03,0.00,0.00,-0.25,0.84,0.00,-0.25,-0.45,-0.01,-0.69,-0.50,-0.92,0.06,0.11,-0.72,-0.22
50%,5401.50,0.07,0.00,0.41,1.04,0.93,0.00,1.04,0.39,0.04,0.06,-0.04,-0.01,0.52,0.42,0.03,0.08
75%,8101.25,0.16,0.17,0.77,2.26,0.97,0.17,2.26,0.74,0.15,0.76,0.31,0.79,0.95,0.67,0.73,0.33
max,9999.00,1.00,26347.82,1.55,7.55,1.00,26347.82,7.55,0.96,7.55,3.81,29.45,2.54,7.55,29.45,11.55,28.66


### Se hace el merge entre la tabla de Dexport Dummy con la tabla ogi/distancia 2014

In [153]:
# t_dexport_rca = pd.merge(t_Distance2014, t_rca_2014[['country_name','product_code','Dexport_dummy']], how="inner", on=['country_name','product_code']).copy()
t_dexport_ogi_rca = pd.merge(t_ogi2014, t_rca_2014[['country_name','product_code','Dexport_dummy']], how="inner", on=['country_name','product_code']).copy()

t_dexport_ogi_rca=t_dexport_ogi_rca[['country_name','product_code','distancia_2014','rca_2014','pci_2014','Dexport_dummy','ogi_2014','ogi_2014_v2','ogi_2014_norm','ogi_2014_norm_v2']].sort_values(by=['country_name','product_code']).copy()
t_dexport_ogi_rca = t_dexport_ogi_rca.reset_index(drop=True).copy()
t_dexport_ogi_rca.head(n=5)
t_dexport_ogi_rca.sample(n=5)

,country_name,product_code,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2
0,Afghanistan,101,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26
1,Afghanistan,102,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06
2,Afghanistan,103,0.96,0.00,2.25,0,0.85,0.29,0.58,-0.04
3,Afghanistan,104,0.91,0.00,-2.32,0,-0.32,0.23,-1.34,-0.10
4,Afghanistan,105,0.95,0.00,0.75,0,0.59,0.62,0.15,0.28


,country_name,product_code,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2
10979,Antigua and Barbuda,8542,0.98,0.04,2.75,0,0.90,0.06,0.65,-0.27
211182,Saint Barthélemy,8513,0.98,0.00,2.51,0,0.46,-0.29,-0.06,-0.61
227509,Singapore,1001,0.89,0.04,0.25,0,0.13,0.11,-0.61,-0.21
10810,Antigua and Barbuda,8003,0.97,0.00,1.38,0,0.47,0.25,-0.05,-0.08
18864,Bahamas,2908,0.97,0.00,3.51,0,1.11,-0.07,1.01,-0.40


### Tabla con periodos 2015 - 2018

In [152]:
years = [2014,2015,2016, 2017, 2018]
prepotres = df_complexity[df_complexity['year'].isin(years)].copy()
prepotres = prepotres[['country_name','product_code','export_value','rca','year']].sort_values(by=['year','country_name','product_code']).copy()

prepotres = prepotres.reset_index(drop=True)
prepotres.head()
print(prepotres['country_name'].nunique())
prepotres.info()

,country_name,product_code,export_value,rca,year
0,Afghanistan,101,0,0.00,2014
1,Afghanistan,102,187317,0.38,2014
2,Afghanistan,103,0,0.00,2014
3,Afghanistan,104,0,0.00,2014
4,Afghanistan,105,0,0.00,2014


236
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440607 entries, 0 to 1440606
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   country_name  1440607 non-null  object 
 1   product_code  1440607 non-null  int64  
 2   export_value  1440607 non-null  int64  
 3   rca           1440607 non-null  float64
 4   year          1440607 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 55.0+ MB


### Product Appereance

In [154]:
t_prodApp =  pd.merge(prepotres, t_dexport_ogi_rca[['country_name','product_code','distancia_2014','rca_2014','pci_2014','Dexport_dummy','ogi_2014','ogi_2014_v2','ogi_2014_norm','ogi_2014_norm_v2']], how="inner", on=['product_code','country_name']).copy()
t_prodApp.head(n=10)
# t_prodApp.sample(n=6)

print(t_prodApp['country_name'].nunique())

,country_name,product_code,export_value,rca,year,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2
0,Afghanistan,101,0,0.00,2014,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26
1,Afghanistan,101,0,0.00,2015,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26
2,Afghanistan,101,0,0.00,2016,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26
3,Afghanistan,101,0,0.00,2017,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26
4,Afghanistan,101,0,0.00,2018,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26
5,Afghanistan,102,187317,0.38,2014,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06
6,Afghanistan,102,0,0.00,2015,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06
7,Afghanistan,102,64743,0.13,2016,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06
8,Afghanistan,102,417978,0.84,2017,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06
9,Afghanistan,102,130333,0.22,2018,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06


235


In [155]:
t_pApp =t_prodApp.copy()
t_pApp['rca_t_1'] = t_pApp.groupby(['country_name','product_code'])['rca'].shift(+1)
t_pApp=t_pApp[t_pApp.year != 2014].copy()
t_pApp.info()
t_pApp.head(10)

# t_pApp_mozam = t_pApp[t_pApp['country_name']=='Mozambique'].copy()
# t_pApp_mozam.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1147993 entries, 1 to 1438452
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   country_name      1147993 non-null  object 
 1   product_code      1147993 non-null  int64  
 2   export_value      1147993 non-null  int64  
 3   rca               1147993 non-null  float64
 4   year              1147993 non-null  int64  
 5   distancia_2014    1147993 non-null  float64
 6   rca_2014          1147993 non-null  float64
 7   pci_2014          1147993 non-null  float64
 8   Dexport_dummy     1147993 non-null  int64  
 9   ogi_2014          1147993 non-null  float64
 10  ogi_2014_v2       1147993 non-null  float64
 11  ogi_2014_norm     1147993 non-null  float64
 12  ogi_2014_norm_v2  1147993 non-null  float64
 13  rca_t_1           1147993 non-null  float64
dtypes: float64(9), int64(4), object(1)
memory usage: 131.4+ MB


,country_name,product_code,export_value,rca,year,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2,rca_t_1
1,Afghanistan,101,0,0.00,2015,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26,0.00
2,Afghanistan,101,0,0.00,2016,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26,0.00
3,Afghanistan,101,0,0.00,2017,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26,0.00
4,Afghanistan,101,0,0.00,2018,0.94,0.00,0.09,0,0.38,0.59,-0.20,0.26,0.00
6,Afghanistan,102,0,0.00,2015,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06,0.38
7,Afghanistan,102,64743,0.13,2016,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06,0.00
8,Afghanistan,102,417978,0.84,2017,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06,0.13
9,Afghanistan,102,130333,0.22,2018,0.92,0.38,-0.81,1,0.02,0.39,-0.79,0.06,0.84
11,Afghanistan,103,0,0.00,2015,0.96,0.00,2.25,0,0.85,0.29,0.58,-0.04,0.00
12,Afghanistan,103,0,0.00,2016,0.96,0.00,2.25,0,0.85,0.29,0.58,-0.04,0.00


In [156]:
def cal_product_appearances(data,
    ):
  t_df = data.copy()
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de product appearances...")

  t_df['product_appearance']=t_df.apply( lambda x: 1 if (x.rca_t_1<1 and x.rca>1) else 0,axis = 1)
  return t_df

In [157]:
t_product_appearance= cal_product_appearances(t_pApp)

Se realiza el cálculo de product appearances...


In [158]:
t_product_appearance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1147993 entries, 1 to 1438452
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   country_name        1147993 non-null  object 
 1   product_code        1147993 non-null  int64  
 2   export_value        1147993 non-null  int64  
 3   rca                 1147993 non-null  float64
 4   year                1147993 non-null  int64  
 5   distancia_2014      1147993 non-null  float64
 6   rca_2014            1147993 non-null  float64
 7   pci_2014            1147993 non-null  float64
 8   Dexport_dummy       1147993 non-null  int64  
 9   ogi_2014            1147993 non-null  float64
 10  ogi_2014_v2         1147993 non-null  float64
 11  ogi_2014_norm       1147993 non-null  float64
 12  ogi_2014_norm_v2    1147993 non-null  float64
 13  rca_t_1             1147993 non-null  float64
 14  product_appearance  1147993 non-null  int64  
dtypes: float64(9), 

###We run the regression only on products that were absent in 2014. 

In [159]:
my_lambda = lambda x: x < 1
# t_pro_app_absent = t_product_appearance.loc[t_product_appearance['rca_2014'].apply(my_lambda)]
t_pro_app_absent = t_product_appearance.loc[t_product_appearance['rca_2014']<1]

t_pro_app_absent.info()
t_product_appearance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024763 entries, 1 to 1438452
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   country_name        1024763 non-null  object 
 1   product_code        1024763 non-null  int64  
 2   export_value        1024763 non-null  int64  
 3   rca                 1024763 non-null  float64
 4   year                1024763 non-null  int64  
 5   distancia_2014      1024763 non-null  float64
 6   rca_2014            1024763 non-null  float64
 7   pci_2014            1024763 non-null  float64
 8   Dexport_dummy       1024763 non-null  int64  
 9   ogi_2014            1024763 non-null  float64
 10  ogi_2014_v2         1024763 non-null  float64
 11  ogi_2014_norm       1024763 non-null  float64
 12  ogi_2014_norm_v2    1024763 non-null  float64
 13  rca_t_1             1024763 non-null  float64
 14  product_appearance  1024763 non-null  int64  
dtypes: float64(9), 

In [160]:
t_pro_app_absent.info()
t_pro_app_absent['product_code'].nunique()
count_product_appe = (t_pro_app_absent['product_appearance'] == 1).sum()
t_pro_app_absent.sample(n=10)
count_product_appe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024763 entries, 1 to 1438452
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   country_name        1024763 non-null  object 
 1   product_code        1024763 non-null  int64  
 2   export_value        1024763 non-null  int64  
 3   rca                 1024763 non-null  float64
 4   year                1024763 non-null  int64  
 5   distancia_2014      1024763 non-null  float64
 6   rca_2014            1024763 non-null  float64
 7   pci_2014            1024763 non-null  float64
 8   Dexport_dummy       1024763 non-null  int64  
 9   ogi_2014            1024763 non-null  float64
 10  ogi_2014_v2         1024763 non-null  float64
 11  ogi_2014_norm       1024763 non-null  float64
 12  ogi_2014_norm_v2    1024763 non-null  float64
 13  rca_t_1             1024763 non-null  float64
 14  product_appearance  1024763 non-null  int64  
dtypes: float64(9), 

1233

,country_name,product_code,export_value,rca,year,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2,ogi_2014_norm,ogi_2014_norm_v2,rca_t_1,product_appearance
699778,Lebanon,5804,196032,0.70,2018,0.73,0.89,0.79,1,0.69,0.35,0.31,0.02,0.89,0
547727,Guinea-Bissau,2520,0,0.00,2016,0.99,0.00,-1.18,0,-0.19,0.93,-1.13,0.58,0.00,0
1202558,Sudan,6504,0,0.00,2018,0.96,0.00,-0.77,0,0.17,0.83,-0.54,0.49,0.00,0
1394984,Venezuela,9005,1517,0.00,2018,0.98,0.00,1.65,0,0.80,0.44,0.49,0.10,0.00,0
1030398,Russian Federation,3906,15339508,0.06,2016,0.92,0.03,3.77,0,1.55,0.81,1.71,0.47,0.05,0
1383188,Vatican City,103,0,0.00,2017,0.89,0.00,2.25,0,0.85,0.59,0.58,0.26,0.00,0
864473,Nauru,3811,0,0.00,2015,0.98,0.00,2.14,0,0.60,-0.00,0.16,-0.33,0.00,0
526564,Grenada,7614,0,0.00,2015,0.95,0.00,0.14,0,0.25,0.50,-0.41,0.17,0.00,0
1230266,Syrian Arab Republic,1508,17129,1.20,2018,0.84,0.15,-1.71,0,-0.21,-0.63,-1.17,-0.95,0.00,1
1142479,Slovenia,6812,9390,0.03,2017,0.73,0.39,-0.51,1,-0.32,-1.35,-1.34,-1.65,0.01,0


17641

In [163]:
t_final_col4 = t_pro_app_absent[['country_name','product_code','year','distancia_2014','pci_2014','product_appearance','ogi_2014','ogi_2014_norm','ogi_2014_v2','ogi_2014_norm_v2']].copy()
t_final_dist_mean = t_final_col4['distancia_2014'].mean()
t_final_dist_std = t_final_col4['distancia_2014'].std()
t_final_pci_mean = t_final_col4['pci_2014'].mean()
t_final_pci_std = t_final_col4['pci_2014'].std()
t_final_col4['dist_2014_norm'] = t_final_col4.apply(lambda x: (x.distancia_2014 - t_final_dist_mean)/t_final_dist_std, axis=1)
t_final_col4['pci_2014_norm'] = t_final_col4.apply(lambda x: (x.pci_2014 - t_final_pci_mean)/t_final_pci_std, axis=1)

t_final_col4.head()

,country_name,product_code,year,distancia_2014,pci_2014,product_appearance,ogi_2014,ogi_2014_norm,ogi_2014_v2,ogi_2014_norm_v2,dist_2014_norm,pci_2014_norm
1,Afghanistan,101,2015,0.94,0.09,0,0.38,-0.20,0.59,0.26,0.35,-0.52
2,Afghanistan,101,2016,0.94,0.09,0,0.38,-0.20,0.59,0.26,0.35,-0.52
3,Afghanistan,101,2017,0.94,0.09,0,0.38,-0.20,0.59,0.26,0.35,-0.52
4,Afghanistan,101,2018,0.94,0.09,0,0.38,-0.20,0.59,0.26,0.35,-0.52
6,Afghanistan,102,2015,0.92,-0.81,0,0.02,-0.79,0.39,0.06,0.19,-1.04


In [164]:
t_final_col4.describe()

,product_code,year,distancia_2014,pci_2014,product_appearance,ogi_2014,ogi_2014_norm,ogi_2014_v2,ogi_2014_norm_v2,dist_2014_norm,pci_2014_norm
count,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00
mean,5339.68,2016.50,0.91,0.99,0.02,0.54,0.06,0.38,0.04,-0.00,-0.00
std,2757.66,1.12,0.10,1.73,0.13,0.61,1.00,0.88,0.87,1.00,1.00
min,101.00,2015.00,0.42,-4.76,0.00,-2.34,-4.66,-9.75,-9.92,-5.05,-3.33
25%,2850.00,2015.00,0.87,-0.19,0.00,0.12,-0.62,0.18,-0.15,-0.41,-0.69
50%,5407.00,2016.00,0.94,1.09,0.00,0.58,0.13,0.44,0.10,0.38,0.06
75%,8109.00,2017.00,0.98,2.31,0.00,0.98,0.79,0.67,0.33,0.73,0.76
max,9999.00,2018.00,1.00,7.55,1.00,4.71,6.90,15.56,14.98,0.97,3.80


## STATS Models in Python

In [81]:
!pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.4 MB/s eta 0:00:00


In [1]:
from linearmodels import PanelOLS

data = t_final_col4.set_index(['country_name','year'])

Y_col = ['product_appearance']
X_col = ['pci_2014_norm','ogi_2014_norm']
fe_col = ['country_name','year']

FE = PanelOLS(data[Y_col], data[X_col],
              entity_effects = True,
              time_effects=True
              )
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True )
             # cluster_time=True)
print(result.summary)


ModuleNotFoundError: ignored

In [107]:
data.head()

product_code  distancia_2014  pci_2014  product_appearance  ogi_2014  ogi_2014_norm  dist_2014_norm  pci_2014_norm
country_name year                                                                                                                    
Afghanistan  2015           101            0.94      0.09                   0      0.42          -0.28            0.35          -0.52
             2016           101            0.94      0.09                   0      0.42          -0.28            0.35          -0.52
             2017           101            0.94      0.09                   0      0.42          -0.28            0.35          -0.52
             2018           101            0.94      0.09                   0      0.42          -0.28            0.35          -0.52
             2015           102            0.92     -0.81                   0     -0.48          -0.61            0.19          -1.04

In [79]:
import statsmodels.api as sm

In [ ]:
df_model =t_final_col1.copy()
Y_col = ['product_appearance']
X_col = ['dist_2014_norm','Dexport_dummy']
fe_col = ['country_name','year']
Y=df_model[Y_col]
X=df_model[X_col]
fe=df_model[fe_col]
X_fe = sm.add_constant(fe)

model = sm.Probit(Y, X )
results2 = model.fit()
print(results2.summary())

Optimization terminated successfully.
         Current function value: 0.435091
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:     product_appearance   No. Observations:              1024763
Model:                         Probit   Df Residuals:                  1024761
Method:                           MLE   Df Model:                            1
Date:                Wed, 19 Apr 2023   Pseudo R-squ.:                  -4.002
Time:                        23:42:06   Log-Likelihood:            -4.4587e+05
converged:                       True   LL-Null:                       -89146.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
dist_2014_norm    -0.7294      0.002   -383.869      0.000      -0.733      -0.726
Dexport_dummy    

#Columna 2 - Product Appearances (2015 - 2018)

In [ ]:
t_f_col2=t_final_col1.copy()
def cal_distanXDexport(data):
  t_distDext = data.copy()
  print("Se realiza el cálculo de Dexport X Distance (2014)")
  t_distDext['distance_Dexport_2014'] = t_distDext.apply(lambda x: x.Dexport_dummy*x.dist_2014_norm, axis=1)
  mean =  t_distDext['distance_Dexport_2014'].mean()
  std =  t_distDext['distance_Dexport_2014'].std()
  t_distDext['distance_X_Dexport_2014']=t_distDext.apply(lambda x: (x.distance_Dexport_2014 - mean)/std, axis=1)
  # t_distDext['distance_Dexport_2014_v2'] = t_distDext['Dexport_dummy']*t_distDext['dist_2014_norm']
  print("Cálculo terminado!!")

  return t_distDext

t_final_col2 = cal_distanXDexport(t_f_col2).copy()

Se realiza el cálculo de Dexport X Distance (2014)
Cálculo terminado!!


In [ ]:
# t_final_col1.describe()

t_final_col2.describe()

,product_code,year,distancia_2014,Dexport_dummy,product_appearance,dist_2014_norm,distance_Dexport_2014,distance_X_Dexport_2014
count,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00
mean,5339.68,2016.50,0.91,0.15,0.02,-0.00,-0.18,-0.00
std,2757.66,1.12,0.10,0.35,0.13,1.00,0.64,1.00
min,101.00,2015.00,0.42,0.00,0.00,-5.05,-5.05,-7.65
25%,2850.00,2015.00,0.87,0.00,0.00,-0.41,-0.00,0.28
50%,5407.00,2016.00,0.94,0.00,0.00,0.38,0.00,0.28
75%,8109.00,2017.00,0.98,0.00,0.00,0.73,-0.00,0.28
max,9999.00,2018.00,1.00,1.00,1.00,0.97,0.95,1.78


In [ ]:
data_2 = t_final_col2.set_index(['country_name','year'])

Y_col2 = ['product_appearance']
X_col2 = ['dist_2014_norm','Dexport_dummy','distance_Dexport_2014']
fe_col2 = ['country_name','year']

FE_2 = PanelOLS(data_2[Y_col2], data_2[X_col2],
              entity_effects = True,
              time_effects=True
              )
              
# Result
result_col_2 = FE_2.fit(cov_type = 'clustered',
             cluster_entity=True )
             # cluster_time=True)
print(result_col_2.summary)
print(result_col_2.conf_int(0.01))
# 0.02--> N:0.0035 ,SN:0.0046     
#|0.06 --N: 0.0048, SN: 0.0066
#>0.08-->N:0.0051, SN:0.0071     
#>0.1 --> N: 0.055,SN: 
#">0.12 --> N: 0.058, SN: 0.0085     
#>0.2--> N: 0.0066,SN:0.0104

                          PanelOLS Estimation Summary                           
Dep. Variable:     product_appearance   R-squared:                        0.0165
Estimator:                   PanelOLS   R-squared (Between):             -1.6017
No. Observations:             1024763   R-squared (Within):               0.0165
Date:                Thu, Apr 20 2023   R-squared (Overall):             -0.0196
Time:                        00:09:53   Log-likelihood                 6.481e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      5724.4
Entities:                         235   P-value                           0.0000
Avg Obs:                       4360.7   Distribution:               F(3,1024522)
Min Obs:                       1218.0                                           
Max Obs:                       4898.0   F-statistic (robust):             473.06
                            

#Librerías de Hidalgo

In [ ]:
!pip install economic-complexity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 41.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [ ]:
import economic_complexity as ecplx

In [ ]:
df_base_2018 = df_base[df_base['year']==2018].copy()

In [ ]:
df_pivot = pd.pivot_table(df_base_2018, index=['country_name'],
                                     columns=['product_code'],
                                     values='export_value')\
             .reset_index()\
             .set_index('country_name')\
             .dropna(axis=1, how="all")\
             .fillna(0)\
             .astype(float)

In [ ]:
df_pivot.head()

product_code,101,102,103,104,105,106,201,202,203,204,205,206,207,208,209,210,301,302,303,304,305,306,307,401,402,403,404,405,406,407,408,409,410,501,502,504,505,506,507,508,510,511,601,602,603,604,701,702,703,704,705,706,707,708,709,710,711,712,713,714,801,802,803,804,805,806,807,808,809,810,811,812,813,814,901,902,903,904,905,906,907,908,909,910,1001,1002,1003,1004,1005,1006,1007,1008,1101,1102,1103,1104,1105,1106,1107,1108,1109,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1301,1302,1401,1404,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1601,1602,1603,1604,1605,1701,1702,1703,1704,1801,1802,1803,1804,1805,1806,1901,1902,1903,1904,1905,2001,2002,2003,2004,2005,2006,2007,2008,2009,2101,2102,2103,2104,2105,2106,2201,2202,2203,2204,2205,2206,2207,2208,2209,2301,2302,2303,2304,2305,2306,2307,2308,2309,2401,2402,2403,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2528,2529,2530,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621,2701,2702,2703,...,8421,8422,8423,8424,8425,8426,8427,8428,8429,8430,8431,8432,8433,8434,8435,8436,8437,8438,8439,8440,8441,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451,8452,8453,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8601,8602,8603,8604,8605,8606,8607,8608,8609,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713,8714,8715,8716,8801,8802,8803,8804,8805,8901,8902,8903,8904,8905,8906,8907,8908,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9201,9202,9205,9206,9207,9208,9209,9301,9302,9303,9304,9305,9306,9307,9401,9402,9403,9404,9405,9406,9502,9503,9504,9505,9506,9507,9508,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9701,9702,9703,9704,9705,9706,9999
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.00,130333.00,0.00,2659886.00,0.00,713231.00,0.00,0.00,0.00,92062.00,0.00,3413.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4139.00,0.00,5569.00,38858.00,5359.00,0.00,0.00,0.00,0.00,10427.00,19082.00,720499.00,0.00,4043205.00,0.00,173128.00,0.00,0.00,0.00,16112.00,0.00,12095.00,481874.00,0.00,254972.00,28416918.00,9461113.00,215111.00,9676.00,7097.00,10843511.00,15304.00,434493.00,149812.00,0.00,314690.00,67298056.00,23522.00,43104.00,67236816.00,2621.00,62823712.00,459527.00,200201264.00,1679515.00,21835542.00,16994596.00,27987412.00,439629.00,884391.00,28342332.00,0.00,0.00,72790.00,867394.00,6794.00,0.00,0.00,0.00,0.00,15419482.00,20157944.00,3184.00,0.00,464393.00,0.00,0.00,0.00,300769.00,83926.00,0.00,6321.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2161.00,459245.00,16853.00,504204.00,0.00,13807.00,13842392.00,0.00,7262882.00,0.00,9722914.00,1980272.00,0.00,1624271.00,78261392.00,5112418.00,195094.00,227643.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,90438.00,0.00,22357.00,0.00,0.00,0.00,

### RCA, ECI, PCI, Proximity, Relatedness, Distance

In [ ]:
rca = ecplx.rca(df_pivot)
ECI, PCI = ecplx.complexity(rca, iterations=20, drop=True)
proximityValues = ecplx.proximity(rca)
relatedness = ecplx.relatedness(rca,proximityValues)
distance= ecplx.distance(rca, proximityValues)
distance.head()

product_code,101,102,103,104,105,106,201,202,203,204,205,206,207,208,209,210,301,302,303,304,305,306,307,401,402,403,404,405,406,407,408,409,410,501,502,504,505,506,507,508,510,511,601,602,603,604,701,702,703,704,705,706,707,708,709,710,711,712,713,714,801,802,803,804,805,806,807,808,809,810,811,812,813,814,901,902,903,904,905,906,907,908,909,910,1001,1002,1003,1004,1005,1006,1007,1008,1101,1102,1103,1104,1105,1106,1107,1108,1109,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1301,1302,1401,1404,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1601,1602,1603,1604,1605,1701,1702,1703,1704,1801,1802,1803,1804,1805,1806,1901,1902,1903,1904,1905,2001,2002,2003,2004,2005,2006,2007,2008,2009,2101,2102,2103,2104,2105,2106,2201,2202,2203,2204,2205,2206,2207,2208,2209,2301,2302,2303,2304,2305,2306,2307,2308,2309,2401,2402,2403,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2528,2529,2530,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621,2701,2702,2703,...,8421,8422,8423,8424,8425,8426,8427,8428,8429,8430,8431,8432,8433,8434,8435,8436,8437,8438,8439,8440,8441,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451,8452,8453,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8601,8602,8603,8604,8605,8606,8607,8608,8609,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713,8714,8715,8716,8801,8802,8803,8804,8805,8901,8902,8903,8904,8905,8906,8907,8908,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9201,9202,9205,9206,9207,9208,9209,9301,9302,9303,9304,9305,9306,9307,9401,9402,9403,9404,9405,9406,9502,9503,9504,9505,9506,9507,9508,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9701,9702,9703,9704,9705,9706,9999
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.94,0.93,0.95,0.89,0.94,0.89,0.93,0.92,0.96,0.90,0.93,0.93,0.94,0.94,0.95,0.95,0.91,0.91,0.90,0.92,0.92,0.90,0.90,0.94,0.92,0.93,0.95,0.94,0.93,0.92,0.95,0.91,0.94,0.91,0.96,0.90,0.96,0.92,0.92,0.92,0.90,0.91,0.93,0.93,0.93,0.92,0.92,0.90,0.87,0.90,0.92,0.91,0.91,0.89,0.90,0.92,0.91,0.92,0.87,0.90,0.90,0.89,0.90,0.89,0.90,0.90,0.89,0.91,0.89,0.89,0.92,0.91,0.90,0.91,0.91,0.89,0.84,0.90,0.93,0.92,0.92,0.91,0.89,0.89,0.93,0.96,0.94,0.94,0.93,0.90,0.89,0.91,0.90,0.91,0.92,0.92,0.95,0.89,0.94,0.93,0.96,0.91,0.87,0.91,0.91,0.95,0.94,0.87,0.92,0.91,0.96,0.88,0.89,0.94,0.91,0.87,0.91,0.91,0.89,0.96,0.93,0.95,0.91,0.95,0.95,0.92,0.91,0.92,0.93,0.92,0.91,0.93,0.95,0.91,0.91,0.92,0.93,0.94,0.94,0.93,0.94,0.94,0.93,0.95,0.92,0.91,0.90,0.94,0.90,0.92,0.91,0.93,0.96,0.95,0.94,0.93,0.94,0.91,0.92,0.92,0.92,0.91,0.91,0.96,0.94,0.91,0.93,0.92,0.90,0.90,0.92,0.92,0.92,0.93,0.93,0.93,0.93,0.92,0.93,0.92,0.94,0.94,0.92,0.92,0.93,0.91,0.90,0.94,0.93,0.91,0.92,0.93,0.92,0.93,0.90,0.92,0.93,0.91,0.94,0.92,0.94,0.92,0.92,0.94,0.92,0.93,0.89,0.92,0

In [ ]:
cogHidal = ecplx.opportunity_gain(rcas=rca,
                                proximities=proximityValues,
                                pci=PCI)

In [ ]:
CogHidalMozam = cogHidal.loc[('Mozambique'),:]
cogHidalMozam = CogHidalMozam.to_frame(name="cog").reset_index()
cogHidalMozam.describe()

,product_code,cog
count,1220.00,1220.00
mean,5297.65,0.18
std,2766.50,0.14
min,101.00,-0.16
25%,2843.75,0.07
50%,5356.00,0.18
75%,8104.25,0.28
max,9999.00,0.72


#### OGI Y COG MANUAL

In [ ]:
rcas = rca.copy()
rcas[rcas >= 1] = 1
rcas[rcas <1] = 0
inverse_rcas = 1 - rcas
prox_ratio = proximityValues / proximityValues.sum()
middle = inverse_rcas.multiply(PCI)
opp_gain=middle.dot(prox_ratio)
Ogi = opp_gain
OgiMozam = Ogi.loc[('Mozambique'),:]

ogi = OgiMozam.to_frame(name="ogi").reset_index()
# ogi.head()
ogi.describe()

,product_code,ogi
count,1220.00,1220.00
mean,5297.65,0.16
std,2766.50,0.20
min,101.00,-0.50
25%,2843.75,0.02
50%,5356.00,0.18
75%,8104.25,0.31
max,9999.00,0.80


In [ ]:
dcp = ecplx.relatedness(inverse_rcas,proximityValues )
    # now get the inverse
dcp = 1 - dcp
    # we now have the right-half of the equation
right = dcp.multiply(PCI)
# right.head()
cog = opp_gain + right
cog.head()

product_code,101,102,103,104,105,106,201,202,203,204,205,206,207,208,209,210,301,302,303,304,305,306,307,401,402,403,404,405,406,407,408,409,410,501,502,504,505,506,507,508,510,511,601,602,603,604,701,702,703,704,705,706,707,708,709,710,711,712,713,714,801,802,803,804,805,806,807,808,809,810,811,812,813,814,901,902,903,904,905,906,907,908,909,910,1001,1002,1003,1004,1005,1006,1007,1008,1101,1102,1103,1104,1105,1106,1107,1108,1109,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1301,1302,1401,1404,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1601,1602,1603,1604,1605,1701,1702,1703,1704,1801,1802,1803,1804,1805,1806,1901,1902,1903,1904,1905,2001,2002,2003,2004,2005,2006,2007,2008,2009,2101,2102,2103,2104,2105,2106,2201,2202,2203,2204,2205,2206,2207,2208,2209,2301,2302,2303,2304,2305,2306,2307,2308,2309,2401,2402,2403,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2528,2529,2530,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621,2701,2702,2703,...,8421,8422,8423,8424,8425,8426,8427,8428,8429,8430,8431,8432,8433,8434,8435,8436,8437,8438,8439,8440,8441,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451,8452,8453,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8601,8602,8603,8604,8605,8606,8607,8608,8609,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713,8714,8715,8716,8801,8802,8803,8804,8805,8901,8902,8903,8904,8905,8906,8907,8908,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9201,9202,9205,9206,9207,9208,9209,9301,9302,9303,9304,9305,9306,9307,9401,9402,9403,9404,9405,9406,9502,9503,9504,9505,9506,9507,9508,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9701,9702,9703,9704,9705,9706,9999
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.15,-0.02,0.24,-0.32,0.08,-0.35,-0.01,-0.15,0.35,-0.35,0.01,-0.01,0.10,0.07,0.24,0.23,-0.25,-0.35,-0.47,-0.32,-0.35,-0.57,-0.42,0.06,-0.14,0.01,0.25,0.13,0.06,-0.08,0.21,-0.25,0.06,-0.02,0.21,-0.16,0.30,0.04,-0.17,-0.16,-0.39,-0.18,0.03,-0.07,-0.25,-0.15,-0.04,-0.27,-0.45,-0.13,-0.03,-0.14,-0.16,-0.49,-0.35,-0.14,-0.21,-0.12,-0.56,-0.45,-0.56,-0.40,-0.50,-0.55,-0.30,-0.17,-0.47,-0.04,-0.30,-0.33,-0.08,-0.21,-0.17,-0.29,-0.31,-0.53,-0.62,-0.43,-0.21,-0.22,-0.34,-0.43,-0.26,-0.42,0.01,0.30,0.13,0.15,-0.08,-0.51,-0.43,-0.18,-0.37,-0.26,-0.17,-0.12,0.32,-0.43,0.13,0.03,0.39,-0.16,-0.62,-0.28,-0.09,0.16,0.05,-0.57,-0.12,-0.16,0.33,-0.53,-0.45,0.11,-0.02,-0.71,-0.19,-0.19,-0.44,0.36,0.03,0.27,-0.47,0.23,0.27,-0.18,-0.28,-0.07,0.03,-0.50,-0.18,-0.36,0.24,-0.15,-0.33,-0.23,-0.02,0.12,0.09,-0.18,0.05,0.10,0.02,0.24,-0.35,-0.41,-0.44,0.09,-0.34,-0.11,-0.56,-0.44,0.23,0.09,0.02,0.04,0.05,-0.25,-0.21,-0.07,-0.06,-0.19,-0.22,0.27,0.19,-0.17,-0.04,-0.08,-0.30,-0.28,-0.09,-0.11,-0.12,-0.01,-0.01,0.00,-0.05,-0.08,-0.03,-0.06,0.11,0.06,-0.17

In [ ]:
CogMozam = cog.loc[('Mozambique'),:]
cogMozam = CogMozam.to_frame(name="cog").reset_index()
moz2018=moz2018.sort_values(by=['product_code'])
eci = ECI.to_frame(name="ECI").reset_index()
pci = PCI.to_frame(name="PCI").reset_index()

eciMozambique=eci[eci['country_name']=='Mozambique']
cogMozam.head()
moz2018.head()
cogHidalMozam.head()
eciMozambique.head()
pci.describe()

NameError: ignored

### Se descarga el dataframe de ECI y PCI a la máquina: 


In [ ]:
# from google.colab import files
# df_ecm2.to_csv('df_ecm3.csv') 
# files.download('df_ecm3.csv')